<H2><CENTER>Performing Relational Operations and Linear Algebra Back-and-forth using AIDA on TPC-H</CENTER></H2>

In this workflow, we will demonstrate how we can use AIDA's TabularData objects to perform relational operations and linear algebra on the data residing an RDBMS seamlessly and how it is possible to go back-and-forth between the two without any specific effort from the user.

<HR>
Once we import AIDA components, we are good to go !

In [1]:
from aida.aida import *;
import numpy as np;

Let us start by establishing a connection to the database and get a connection object. This is syntactically very similar to any JDBC/ODBC based applications.

In [2]:
host='cerberus'; dbname='demo'; user='demo'; passwd='demo'; jobName='TPCHExplorer-RA-LA-RA'; port=55660;
dw = AIDA.connect(host, dbname, user, passwd, jobName, port);

Let us find the number of items associated with a particular percentage of discount from the lineitem table. This is essentially a relational aggregation operation and can be expressed using AIDA's ORM syntax over the TabularData object.

In [3]:
l = dw.lineitem;
d = l.aggregate(('l_discount', {SUM('l_quantity'):'total_quantity'}),('l_discount',));
print(d.cdata);

OrderedDict([('l_discount', array([0.04, 0.09, 0.1 , 0.07, 0.  , 0.06, 0.01, 0.03, 0.02, 0.08, 0.05])), ('total_quantity', array([13904583., 13904479., 13912223., 13938963., 13910779., 13902806.,
       13922569., 13923209., 13937010., 13877240., 13944934.]))])


Using linear algebra, we will slighltly transform this data so that the discounts are whole numbers and the quantity is expressed in millions.

At this point, behind the scene, AIDA hands over the data of the TabularData object (from the RDBMS) to numpy modules to perform this computation. Using techniques such as zero-copy results in negligible ovehead for such handovers.

In [4]:
d2 = d * np.asarray([[100, 0.000001]]);
print(d2.cdata)

OrderedDict([('l_discount', array([ 4.,  9., 10.,  7.,  0.,  6.,  1.,  3.,  2.,  8.,  5.])), ('total_quantity', array([13.904583, 13.904479, 13.912223, 13.938963, 13.910779, 13.902806,
       13.922569, 13.923209, 13.93701 , 13.87724 , 13.944934]))])


Next we chose to select only the data where the discount is 5% or above. This can be performed as a relational operation on the TabularData object using the ORM syntax.

Behind the scenes, AIDA now hands over this data (from numpy) to the RDBMS to execute a query with a selection condition attached to it.

In [5]:
d3 = d2.filter(Q('l_discount', C(5), CMP.GTE));
print(d3.cdata);

OrderedDict([('l_discount', array([ 9., 10.,  7.,  6.,  8.,  5.])), ('total_quantity', array([13.904479, 13.912223, 13.938963, 13.902806, 13.87724 , 13.944934]))])


In [6]:
dw._close();
dw=None;